In [1]:
import dc_stat_think as dc
import pandas as pd
import numpy as np


p = r'E:\db\CLUSTERS_DONE.csv'
df = pd.read_csv(p)

In [2]:
df.head()

,mfr,cv isi,recording_id,neuron_id,var,isi_var,minute_gap,hc_cluster,colors
0,5.928903,0.239206,1,1,0.172195,1.999181e+06,0,Clocklike Slow Regular,#c44e52
1,1.206947,0.787332,1,2,1.174466,3.233078e+11,1,Slow Irregular,#4c72b0
2,6.598349,0.299881,1,3,0.244312,2.065885e+06,0,Clocklike Slow Regular,#c44e52
3,10.164469,0.332762,1,4,9.095547,5.526932e+06,0,Fast Varied,#55a868
4,8.665576,0.745341,1,5,8.107467,1.013471e+07,0,Fast Varied,#55a868


In [3]:
clusters = df['hc_cluster'].unique()

In [4]:
c = clusters[0]
col = 'mfr'
data = df.loc[df['hc_cluster']==c][col].values

In [5]:
type(df.loc[df['hc_cluster']==c][col])

pandas.core.series.Series

In [6]:
reps = dc.draw_bs_reps(data=data, func=np.mean, size=10000)

In [7]:
np.mean(reps)
np.percentile(reps, [0.05, 0.95])

array([3.27670502, 3.3710778 ])

In [8]:
def bs_percentile(column):
    if isinstance(column, pd.core.series.Series):
        column =  column.values
    reps = dc.draw_bs_reps(data=column, func=np.mean, size=100000)
    return np.percentile(reps, [0.05, 0.95])

In [9]:
mfr = df.groupby('hc_cluster')['mfr'].apply(bs_percentile).apply(pd.Series).apply(np.round, decimals=3)

In [10]:
mfr.columns = ['Lower Bound', 'Upper Bound']

In [11]:
mfr.index.name = 'Cluster'

In [12]:
mfr

,Lower Bound,Upper Bound
Cluster,,
Clocklike Slow Regular,3.276,3.372
Fast Varied,12.014,12.545
Slow Irregular,1.786,1.889
Very Fast Regular,29.986,31.487


In [13]:
cv_isi = df.groupby('hc_cluster')['cv isi'].apply(bs_percentile).apply(pd.Series).apply(np.round, decimals=3)

In [14]:
cv_isi.columns = ['Lower Bound', 'Upper Bound']

In [15]:
cv_isi.index.name = 'Cluster'

In [16]:
df_report = pd.concat([mfr, cv_isi], axis=1)

In [17]:
columns=[('Mean Firing Rate [Hz]','Lower bound'),('Mean Firing Rate [Hz]','Upper bound'), 
         ('ISI Coefficient of Variation','Lower bound'), ('ISI Coefficient of Variation','Upper bound')]

df_report.columns=pd.MultiIndex.from_tuples(columns)

In [18]:
df_report

Mean Firing Rate [Hz]              \
                                 Lower bound Upper bound   
Cluster                                                    
Clocklike Slow Regular                 3.276       3.372   
Fast Varied                           12.014      12.545   
Slow Irregular                         1.786       1.889   
Very Fast Regular                     29.986      31.487   

                       ISI Coefficient of Variation              
                                        Lower bound Upper bound  
Cluster                                                          
Clocklike Slow Regular                        0.405       0.412  
Fast Varied                                   0.388       0.412  
Slow Irregular                                0.803       0.811  
Very Fast Regular                             0.255       0.274

In [19]:
import os


out_dir = r'E:\db'
df_report.to_excel(os.path.join(out_dir, 'cluster_stats.xlsx'))